# The Renormalization Group (RG) method #
##  for weakly nonlinear oscillator equations ##

Support material for the book __Perturbation Methods from a Backward Error Point of View__ by Robert M. Corless and Nicolas Fillion, in preparation.

This section of the book follows the method given in the very nice 2012 SIAM Review (Education) paper ["The Renormalization Group: A Perturbation Method for the Graduate Curriculum"](https://doi.org/10.1137/080731967) by Eleftherios Kirkinis. 


Copyright (c) 2024 Robert M. Corless

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.



## Preliminaries ##

First, some setup: a macro to allow easy typing of $\varepsilon$, and some helper utilities.

In [1]:
starting_time := time():
macro( e = varepsilon );

kilobytes used=1798, alloc=5424, time=0.36

$$\varepsilon$$

The following helper routine makes life somewhat easier.

"partsol" finds a particular solution of $y'' + y = Q \exp(i t\omega)$ more quickly than dsolve does, and leaves the answer in the form $P\exp(i t\omega)$.


In [2]:
partsol := proc( Q, x, omega)
	local k, m, mdeg, p;
	m := degree( Q, x );
	if omega^2=1 then
		mdeg := m+1;
	else
		mdeg := m;
	end if;
	P := add(p[k]*x^k,k=0..mdeg);
	zr := collect( Q - (1-omega^2)*P - 
	              2*I*omega*diff(P,x) - diff(P,x,x), x);
	eqs := PolynomialTools:-CoefficientList(zr,x);
	sol := solve(convert(eqs,set), {seq(p[k],k=0..mdeg)} );
	return eval( eval(P, sol), p[0]=0 );
end proc:

Just to see how it works, here are three examples.  First, solve $y'' + y = 1*\exp(i 1 x)$:

In [3]:
a := partsol( 1, x, 1 );

$$-\frac{\mathrm{I}}{2} x$$

Now we check, to see if $y = a\exp(ix)$ solves the given differential equation:

In [4]:
y := a*exp(I*x):
residual := diff(y,x,x) + y - exp(I*x);

$$0$$

Another example: solve $y'' + y = 1\cdot \exp(i 2 x)$:

In [5]:
a := partsol( 1, x, 2 );

kilobytes used=3879, alloc=9776, time=0.59

$$-{\frac{1}{3}}$$

In [6]:
y := a*exp(I*2*x):
residual := diff(y,x,x) + y - exp(I*2*x);

$$0$$

Both of those checked out.  Now try a harder example, $y'' + y = (x+3)\exp(ix)$, which has some resonance.

In [7]:
a := partsol( x+3, x, 1 );

$$-\frac{\mathrm{I} x^{2}}{4}+\left(\frac{1}{4}-\frac{3 \,\mathrm{I}}{2}\right) x$$

In [8]:
y := a*exp(I*x):
residual := diff(y,x,x) + y - (x+3)*exp(I*x);

$$-\frac{\mathrm{I} \,{\mathrm e}^{\mathrm{I} x}}{2}+2 \,\mathrm{I} \left(-\frac{\mathrm{I} x}{2}+\frac{1}{4}-\frac{3 \,\mathrm{I}}{2}\right) {\mathrm e}^{\mathrm{I} x}-\left(x +3\right) {\mathrm e}^{\mathrm{I} x}$$

That doesn't _look_ like zero, but it is.

In [9]:
simplify( residual );

$$0$$

## Input parameters: the DE and the order we work to ##

This example is the Rayleigh equation.  You can change it to be whatever you want.

In [10]:
y := 'y';
firstorder := (diff(v(t),t))^2 - 2*v(t) + v(t)^2 - e*v(t)^3 + beta;

                                    y := y

$$\left(\frac{d}{d t}v \left(t \right)\right)^{2}-2 v \left(t \right)+v \left(t \right)^{2}-\varepsilon  v \left(t \right)^{3}+\beta$$

In [11]:
latex(firstorder);

\left(\frac{d}{d t}v \left(t \right)\right)^{2}-2 v \left(t \right)+v \left(t \right)^{2}-\varepsilon  v \left(t \right)^{3}+\beta

In [12]:
secondorder := normal( diff( firstorder, t )/2/diff(v(t),t));

$$-\frac{3 v \left(t \right)^{2} \varepsilon}{2}+\frac{d^{2}}{d t^{2}}v \left(t \right)+v \left(t \right)-1$$

In [13]:
latex(secondorder);

-\frac{3 v \left(t \right)^{2} \varepsilon}{2}+\frac{d^{2}}{d t^{2}}v \left(t \right)+v \left(t \right)-1

In [14]:
Jupyter:-SetOutputRendererByType( algebraic, "text/latex" );
y := 'y':
residual := 'residual':
WeaklyNonlinear := eval(secondorder, v(t)=y(t)+1);

                                    default

Choose the order to work to.  We will get the solution to $O(\varepsilon^{N+1})$, first with secularity, and then without.  We have taken $N$ as high as $16$, where the total process took about twenty-six minutes with our original, specialized code.  We have since replaced that specialized code because it was too fragile; changing the differential equation caused it to break very easily.  The timing is now longer: for $N=8$ it takes 50 seconds, or less than a minute. For N=12 it takes 336 seconds, or five and a half minutes. For $N=16$ the code now takes 35 minutes.

In [15]:
N := 2;  # Taking N=0 breaks some assumptions below, so choose N > 0.

                                    N := 2

## Compute the naive secular solution ##

In [16]:
ys := Array(0 .. N): # Placeholder for secular solution terms
rs := Array(0 .. N): # Placeholder for residuals

The initial approximation is a solution to $y'' + y = 0$, which we write in complex conjugate form.

In [17]:
ys[0] := A*exp(phi*I)*exp(t*I) + A*exp(-I*phi)*exp(-I*t);

In [18]:
zs := ys[0]: # Current secular solution in the variable "zs"

Build up the regular perturbation solution using our fast "particular solution" solver. We ignore initial conditions entirely; we will fix them up at the end if we need to.

Regular perturbation solves the linearized system $y'' + y =$ the negative of the coefficient of $\varepsilon^k$ in the residual of $y_{k-1}$.


In [19]:
loop_start := time():
times := Array(1 .. N):
for k to N do
    residuals[k - 1] := combine(expand(
                          coeff(eval(WeaklyNonlinear, y(t) = zs), e, k)),
                          exp);
    if type(residuals[k - 1], `+`) then
        newans := 0;
        for trm in op(residuals[k - 1]) do
            expco := eval(trm, exp = 1);
            exptrm := simplify(trm/expco) assuming A>0, phi::real ;
            if not match(exptrm = 'exp((t + theta)*omega__1*I)', t, 's') then
                error "Unexpected form of a term in the residual:", exptrm;
            else
                newans := newans + partsol(op(eval([expco, t, omega__1], s)))*eval(exp((t + theta)*omega__1*I), s);
            end if;
        end do;
    end if;
    ys[k] := -newans;
    zs := zs + ys[k]*e^k;
    times[k] := time();
end do:

Always compute the final residual.  Here, we are just computing the leading coefficient of the final residual; that is enough for us here.

In [20]:
residuals[N] := CodeTools:-Usage(
  map(LargeExpressions:-Veil[K], 
  series(leadterm(
    combine(expand(eval(WeaklyNonlinear, y(t) = zs)), exp)),
       e)
     )                          );

memory used=15.61MiB, alloc change=28.00MiB, cpu time=281.00ms, real time=255.00ms, gc time=78.12ms

The secular term has $\exp(i t)$ in it, but might also get mixed up with $\exp(i(t+\phi))$ or $\exp(i(t-\phi))$.  So we get them all.

In [21]:
saecular := coeff(zs, exp(t*I)) 
  + exp(phi*I)*coeff(zs, exp((t + phi)*I)) 
  + exp(-I*phi)*coeff(zs, exp((t - phi)*I)):

We clean this up a bit.

In [22]:
saecular := (collect(saecular, e, m -> Re(m) + Im(m)*I) 
             assuming (0 < t, A>0, phi::real)):
saecular := (simplify(saecular) assuming (0 < t, A > 0, phi::real)):
saecular := collect(saecular, e, factor):

## Compute the secular series ##

The _secular series_ is available from this term.

In [23]:
yA := collect(saecular/(A*exp(phi*I)), e):
yA := (collect(yA, e, m -> Re(m) + Im(m)*I) 
  assuming (0 < t, A>0, phi::real)):
yA := combine(yA, trig):

## Express A in terms of R ##
Now we will need to invert $R = A y_A$ and the best way is by a real equation.  $R$ is real and $A > 0$ by our choice of $\phi$.  The equation that we will solve is $f(R,A,\varepsilon) = 0$ where $f(R,A,\varepsilon) = R^2 - A^2(\Re(y_A)^2 + \Im(y_A)^2)$.

In [24]:
rhosq := series(evalc(Re(yA)^2 + Im(yA)^2), e, N + 1):
rhosq := map(simplify, rhosq):
rhosq := convert(rhosq, polynom):
freqn := R^2 - A^2*rhosq:

We will solve this perturbatively for $A = g(R,\varepsilon)$.  We need the derivative, for our regular perturbation method.

In [25]:
eval(diff(freqn, R), e = 0);

So our inverse operator is $\frac{1}{2R}$.  Now check that our initial approximation A=R is accurate enough:

In [26]:
series(leadterm(eval(freqn, A = R)), e);

So $A =  R + O(\varepsilon)$ is a good initial approximation.
Let's use the array "Eh" to represent our terms in the $A$ expansion.

In [27]:
Eh := Array(0 .. N): # get them one by one
residEh := Array(0 .. N): # keep the residuals as well

Our initial approximation:

In [28]:
Eh[0] := R:
Ehz := Eh[0]: # Running solution

Now we use our basic perturbation algorithm to find $A$ in terms of $R$, $\varepsilon$, and $t$.

In [29]:
for k to N do
    residEh[k - 1] := coeff(map(simplify, series(eval(freqn, A = Ehz), e, k + 2)), e, k);
    Eh[k] := residEh[k - 1]*e^k/(2*R);
    Ehz := Ehz + Eh[k];
end do:

Check the final residual: (We don't want to look at the leading coefficient, which will be ugly, but we do want to know the order: so we Veil the coefficient)

In [30]:
residEh[N] := CodeTools:-Usage(
  map(LargeExpressions:-Veil[K], 
  map(simplify, 
  series(eval(freqn, A = Ehz), e, N + 2)
     )
     )                        );

memory used=3.41KiB, alloc change=0 bytes, cpu time=0ns, real time=1000.00us, gc time=0ns

In [31]:
yA := convert(yA, polynom): # The series data structure can be inconvenient
# Maple's internal series data structure is a bit hard to handle sometimes.
coeff( yA, e, 1 ); # Let's look at the first term.

                                   -3/2 I t

## Reverse Engineering the Amplitude and Phase DEs ##

The real part of $\dot{y}_A/y_A$ is $\dot{R}/R$, and this is crucial for the renormalization process.

In [32]:
reyAtyA := (Re(diff(yA, t)/yA) assuming (0 < t, 0 < e, 0 < A, phi::real)):
series(eval(reyAtyA, A = Ehz), e, N + 1):
simplify(%):
slowRderiv := R*convert(%,polynom);

Now we convert that _expression_ into a _procedure_ which Maple will be able to use to differentiate the function $R(t)$.

In [33]:
`diff/R` := codegen[makeproc](
  eval(slowRderiv*'diff'('expr', 'var'), R = R(expr)), 
  parameters = [expr, var]);

proc (expr, var) 0 end proc

Now test that Maple knows what the derivative of $R(t)$ is.

In [34]:
diff( R(t), t );

Now do the same for the $\theta$ equation

In [35]:
ImyAtyA := (Im(diff(yA, t)/yA) assuming (0 < t, 0 < e, 0 < A, phi::real )):
ImyAtyA := series(eval(ImyAtyA, A = Ehz), e, N + 1):
ImyAtyA := simplify(ImyAtyA):
ImyAtyA := convert(ImyAtyA, polynom):
`diff/theta` := codegen[makeproc](
  eval(ImyAtyA*'diff'('expr', 'var'), R = R(expr)), 
  parameters = [expr, var]):

In [36]:
diff( theta(t), t );

In [37]:
latex( diff(theta(t),t));

-\frac{3 \varepsilon}{2}+\left(-\frac{15 R \left(t \right)^{2}}{4}-\frac{27}{8}\right) \varepsilon^{2}

In [43]:
z := 2*R(t)*cos(t+theta(t));
loop_start := time():
times := Array(1 .. N):
for k to N do
    residuals[k - 1] := combine(expand(
                          coeff(eval(WeaklyNonlinear, y(t) = z), e, k)),
                          exp);
    if type(residuals[k - 1], `+`) then
        newans := 0;
        for trm in op(residuals[k - 1]) do
            expco := eval(trm, exp = 1);
            exptrm := simplify(trm/expco) assuming A>0, phi::real ;
            if not match(exptrm = 'exp((t + theta)*omega__1*I)', t, 's') then
                error "Unexpected form of a term in the residual:", exptrm;
            else
                newans := newans + partsol(op(eval([expco, t, omega__1], s)))*eval(exp((t + theta)*omega__1*I), s);
            end if;
        end do;
    end if;
    ys[k] := -newans;
    z := z + ys[k]*e^k;
    times[k] := time();
end do:

                         z := 2 R(t) cos(t + theta(t))

Maple Error: Error, (in partsol) range bounds in add must be numeric

## Sensitivity (Conditioning) of the DE ##

Our example is an unforced nonlinear oscillator. We have shown that the renormalization method gets an exact solution to a problem that is uniformly near to the original problem, and by making $\varepsilon$ small enough we can ensure that the problem we have solved is as close as we like to the one that we started to solve.  As usual, we have to think about what the effects of small changes to the problem are.

Also as usual for weakly nonlinear oscillators, the problem _is sensitive_ to changes that are resonant.  That is the whole purpose of the solution process, to remove spurious resonances which would throw the solution off over long times. At its simplest, the underlying linear problem has the Green's function $\sin(t-\tau)$, as we discuss in the book.  If the perturbing force is nonresonant, then it does not have much effect: the condition number is nearly $1$, in fact.

We can see that in a simple way, here, by merely plotting the difference between the $O(\varepsilon^N)$ solution and the $O(\varepsilon^{N+1})$ solution and comparing it to the residual, for some reasonable value of $\varepsilon$.  We used $\varepsilon = 0.4$ above, which was suitable for quite large $N$.  Comparing the two graphs, we see that the effect of the nonresonant perturbation is _smaller_ than the residual.  For $N=13$ and $\varepsilon=0.4$ the magnitude of the residual is about $4\times 10^{-8}$, while the change in the solution is at most $6 \times 10^{-9}$.

Of course, this only samples the effect of a particular nonresonant forcing; a more general analysis would be more convincing.  Perhaps there is some other nonresonant forcing that would have a greater effect.  But since the kernel of the Green's function integral is bounded by $1$, it can never have _that_ much effect.